In [1]:
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten
from keras.layers import Dense, Dropout
from keras.callbacks import TensorBoard, ModelCheckpoint
import os


Using TensorFlow backend.


In [2]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [3]:
X_train, y_train = load_mnist('../input/mnist_fashion', kind='train')
X_test, y_test = load_mnist('../input/mnist_fashion', kind='t10k')

In [4]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (5, 5), padding="same", input_shape=[28, 28, 1]),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(64, (5, 5), padding="same"),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])


In [5]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              3212288   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
__________

In [12]:
def create_callbacks(name):
    tensorboard_callback = TensorBoard(log_dir=os.path.join(os.getcwd(), "tensorboard_log", name), write_graph=True, write_grads=False)
    checkpoint_callback = ModelCheckpoint(filepath="./model-weights-" + name + ".{epoch:02d}-{val_loss:.6f}.hdf5", monitor='val_loss',
                                          verbose=0, save_best_only=True)
    return [tensorboard_callback]

In [6]:
X_train.shape


(60000, 784)

In [7]:
X_train = X_train.reshape([-1, 28, 28, 1])
X_test = X_test.reshape([-1, 28, 28, 1])
X_train = X_train/255
X_test = X_test/255


In [8]:
y_train = keras.utils.np_utils.to_categorical(y_train)
y_test = keras.utils.np_utils.to_categorical(y_test)


In [14]:
model.compile(keras.optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_split=0.10, batch_size=64, epochs=25, verbose=1, callbacks=create_callbacks("fashion_mnist"))


Train on 54000 samples, validate on 6000 samples
Epoch 1/25
54000/54000 [==============================] - 11s 198us/step - loss: 0.0046 - acc: 0.9985 - val_loss: 0.0318 - val_acc: 0.9925
Epoch 2/25
54000/54000 [==============================] - 10s 193us/step - loss: 0.0043 - acc: 0.9986 - val_loss: 0.0383 - val_acc: 0.9912
Epoch 3/25
54000/54000 [==============================] - 10s 193us/step - loss: 0.0046 - acc: 0.9986 - val_loss: 0.0368 - val_acc: 0.9920
Epoch 4/25
54000/54000 [==============================] - 10s 193us/step - loss: 0.0044 - acc: 0.9984 - val_loss: 0.0363 - val_acc: 0.9913
Epoch 5/25
54000/54000 [==============================] - 10s 192us/step - loss: 0.0034 - acc: 0.9988 - val_loss: 0.0399 - val_acc: 0.9912
Epoch 6/25
54000/54000 [==============================] - 10s 192us/step - loss: 0.0035 - acc: 0.9989 - val_loss: 0.0346 - val_acc: 0.9927
Epoch 7/25
54000/54000 [==============================] - 10s 192us/step - loss: 0.0033 - acc: 0.9991 - val_loss: 0.0

In [10]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)


10000/10000 [==============================] - 1s 83us/step


In [11]:
print(test_loss, test_accuracy)

0.02750000033098677 0.9934


In [15]:
# Save your model weights
model.save("fashion_mnist.h5")